In [1]:
from GBP.data import DataGenerator
from GBP.gbp import run_GaBP_SYNC_ACCELERATED, run_GaBP_HARDWARE_BESTCASE_RESIDUAL, run_GaBP_HARDWARE_BESTCASE, run_GaBP_HARDWARE_ACCELERATED, run_GaBP_HARDWARE_ACCELERATED_RESIDUAL, run_GaBP_HARDWARE_ACCELERATED_EXCLUSION, run_GaBP_HARDWARE_ACCELERATED_EXCLUSION_NEIGHBOURS_ACROSS_STREAMS, run_GaBP_HARDWARE_ACCELERATED_EXCLUSION_NEIGHBOURS_ACROSS_STREAMS_STOCHASTIC
# , run_GaBP_HARDWARE_ACCELERATED_EXCLUSION_NEIGHBOURS_ACROSS_2_STREAMS
from GBP.utilities import HiddenPrints
from GBP.visulisation import set_plot_options, get_plot_colors, NetworkxGraph, AnalyzeResult
 
import warnings
import matplotlib
import numpy as np
import math
import random

# Option 1: Suppress all warnings
warnings.filterwarnings("ignore")

set_plot_options()
colors = get_plot_colors()

data_gen = DataGenerator()
result_analyzer = AnalyzeResult()

In [2]:
# import random

# def custom_sample(items, n):
#     indices = random.sample(range(len(items)), n)
#     return [items[i] for i in indices]

# # Example usage
# items = ["apple", "banana", "orange", "grape", "kiwi"]
# n = 10
# selected_items = custom_sample(items, n)
# print(selected_items)

In [3]:
num_nodes = 1000

sync_convergence_threshold = 1*10**-8

# NODE_UPDT_PE = 137
# PEs = 5

#### Sync

In [4]:
sum_of_iterations = 0
num_iterations = 1

for _ in range(1000):
    # A, b = data_gen.get_1D_line_matrix_PSD(num_nodes)
    # A, b = data_gen.get_1D_line_matrix(num_nodes, scaling=False, normalized=False)
    # A,b = data_gen.get_2D_lattice_matrix_PSD(int(math.sqrt(num_nodes)), int(math.sqrt(num_nodes)))
    # A,b = data_gen.get_2D_lattice_matrix_PSD_diagonal_dominant(int(math.sqrt(num_nodes)), int(math.sqrt(num_nodes)), dominance_factor=5)

    # A,b = data_gen.get_2D_lattice_matrix_PSD_shaped(int(math.sqrt(num_nodes)), int(math.sqrt(num_nodes)),eigenvalue_spread=2, regularization_strength=1e-3)

    A,b = data_gen.get_2D_lattice_matrix_PSD_difficult(int(math.sqrt(num_nodes)), int(math.sqrt(num_nodes)), eigenvalue_spread=1e-3, regularization_strength=1e-2, noise_strength=1e-2)

    # b = 10 * b / 0.01283487483740003
    
    graph = NetworkxGraph(A)
    P_i, mu_i, N_i, P_ii, mu_ii, P_ij, mu_ij, iter_dist, stand_divs, means, iteration = run_GaBP_SYNC_ACCELERATED(A, b, max_iter=200, mae=False, convergence_threshold=sync_convergence_threshold, show=True)
    sum_of_iterations += iteration
    final_mean = list(mu_i)
    final_std = P_i
    if iteration < 100000:
        break
    else:
        print("=========== RESTART ===========")

print(f"AVE. SYNC ITERATIONS = {sum_of_iterations/num_iterations}")

Condition number of A: 366.1066158347566
iteration: 1
2.0850420878068467
-----
iteration: 2
1.7109809373744462
-----
iteration: 3
0.5978025823037695
-----
iteration: 4
0.59480743053946
-----
iteration: 5
0.2043399980405279
-----
iteration: 6
0.1812406899490517
-----
iteration: 7
0.14626726823601982
-----
iteration: 8
0.10211458828749939
-----
iteration: 9
0.08209553014544384
-----
iteration: 10
0.05661945294829112
-----
iteration: 11
0.04649499939727242
-----
iteration: 12
0.03195819433186438
-----
iteration: 13
0.026073108791106032
-----
iteration: 14
0.017937545996340117
-----
iteration: 15
0.0146511004044798
-----
iteration: 16
0.010068894768846232
-----
iteration: 17
0.008233095760720462
-----
iteration: 18
0.005652113968160854
-----
iteration: 19
0.004626473960039804
-----
iteration: 20
0.003172785339119435
-----
iteration: 21
0.0025997824521328114
-----
iteration: 22
0.0017810274368447378
-----
iteration: 23
0.0014609099157923615
-----
iteration: 24
0.000999770510066787
-----
ite

#### Random - No Cache

In [17]:
async_convergence_threshold = 1*10**-5

sum_of_iterations = 0
num_iterations = 10


NODE_UPDT_PE = 84
PEs = 5

for it in range(0,num_iterations):
    # print(f"-------------- ITERATION = {it+1} --------------")
    P_i, mu_i, iteration = run_GaBP_HARDWARE_ACCELERATED(A, b, caching=False, node_updates_per_pe=NODE_UPDT_PE, number_pes=PEs, TRUE_MEAN=final_mean, max_iter=1000, mae=True, convergence_threshold=async_convergence_threshold, show=True)
    sum_of_iterations += iteration
    print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")

print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")

iteration: 0
2.6162409562987343
-----
iteration: 1
2.6162409562987343
-----
2
iteration: 2
2.609959425099677
-----
iteration: 3
2.746201419276231
-----
2
iteration: 4
2.7410799274792375
-----
iteration: 5
2.8200482235097897
-----
2
iteration: 6
2.6907718821546642
-----
iteration: 7
2.8229892036501534
-----
2
iteration: 8
2.651468426399104
-----
iteration: 9
2.5259189089951755
-----
2
iteration: 10
2.279354836427712
-----
iteration: 11
1.9066576999208626
-----
2
iteration: 12
1.8258056078715963
-----
iteration: 13
2.133859887016083
-----
2
iteration: 14
1.3541113644210747
-----
iteration: 15
0.3413478902750794
-----
2
iteration: 16
0.22693611725987545
-----
iteration: 17
0.22163284767481078
-----
2
iteration: 18
0.1721218442285349
-----
iteration: 19
0.16747542189009945
-----
2
iteration: 20
0.14878639154333353
-----
iteration: 21
0.14031829274763488
-----
2
iteration: 22
0.13052259202029518
-----
iteration: 23
0.12544304914985505
-----
2
iteration: 24
0.12010920809191819
-----
iteratio

In [18]:
async_convergence_threshold = 1*10**-5

sum_of_iterations = 0
num_iterations = 10


NODE_UPDT_PE = 84
PEs = 3

for it in range(0,num_iterations):
    # print(f"-------------- ITERATION = {it+1} --------------")
    P_i, mu_i, iteration = run_GaBP_HARDWARE_ACCELERATED_RESIDUAL(A, b, caching=True, node_updates_per_pe=NODE_UPDT_PE, number_pes=PEs, TRUE_MEAN=final_mean, max_iter=1000, mae=True, convergence_threshold=async_convergence_threshold, show=True)
    sum_of_iterations += iteration
    print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")

print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")

iteration: 1
0.8736265494782178
-----
iteration: 2
0.7897621461223806
-----
iteration: 3
0.7169847158777255
-----
iteration: 4
0.6089307207877782
-----
iteration: 5
0.5568441908598233
-----
iteration: 6
0.4725709146149793
-----
iteration: 7
0.3795356795027126
-----
iteration: 8
0.3297984435755458
-----
iteration: 9
0.22639701643741791
-----
iteration: 10
0.1957775568521815
-----
iteration: 11
0.14471460899113275
-----
iteration: 12
0.1274983344808604
-----
iteration: 13
0.10716103431857583
-----
iteration: 14
0.09618050778725243
-----
iteration: 15
0.08419214762679542
-----
iteration: 16
0.07822480430427836
-----
iteration: 17
0.06204215326397956
-----
iteration: 18
0.05843225501643035
-----
iteration: 19
0.047513923159732456
-----
iteration: 20
0.04556722378806072
-----
iteration: 21
0.04145608540553347
-----
iteration: 22
0.040386802644801476
-----
iteration: 23
0.03581153710287272
-----
iteration: 24
0.03511622248656896
-----
iteration: 25
0.02728838616431202
-----
iteration: 26
0.0

In [7]:
async_convergence_threshold = 1*10**-5

sum_of_iterations = 0
num_iterations = 10

for it in range(0,num_iterations):
    node_update_schedule = np.arange(num_nodes, dtype=np.int64)
    np.random.shuffle(node_update_schedule)
    P_i, mu_i, iteration = run_GaBP_HARDWARE_ACCELERATED(A, b, caching=True, mode='fixed', node_update_schedule_enter=node_update_schedule, node_updates_per_pe=NODE_UPDT_PE, number_pes=PEs, TRUE_MEAN=final_mean, max_iter=1000, mae=True, convergence_threshold=async_convergence_threshold, show=True)
    # print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")
    # print(f"streams = {iteration}")
    # print(f"node_update_schedule = {node_update_schedule}")
    sum_of_iterations += iteration

print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")

iteration: 0
2.6162409562987343
-----
iteration: 1
2.6162409562987343
-----
2
iteration: 2
2.7260487731856555
-----
iteration: 3
2.5891369140724696
-----
2
iteration: 4
2.9391803388864988
-----
iteration: 5
2.908451087919616
-----
2
iteration: 6
3.1396379480301375
-----
iteration: 7
3.052232133108148
-----
2
iteration: 8
2.0231460456157615
-----
iteration: 9
1.8123598161214973
-----
2
iteration: 10
0.7858460558999719
-----
iteration: 11
0.6649834548925458
-----
2
iteration: 12
0.5179862722714036
-----
iteration: 13
0.46567848381111243
-----
2
iteration: 14
0.4230981595848423
-----
iteration: 15
0.41561985918156874
-----
2
iteration: 16
0.38236715363508866
-----
iteration: 17
0.27794219152595606
-----
2
iteration: 18
0.2723888235995433
-----
iteration: 19
0.2710793568050986
-----
2
iteration: 20
0.2074157777198667
-----
iteration: 21
0.20357630230599746
-----
2
iteration: 22
0.2019728065368752
-----
iteration: 23
0.17153008964077204
-----
2
iteration: 24
0.1633396710712298
-----
iterati

# RANDOM SHIT

###

#### Random - with Cache

In [8]:
# async_convergence_threshold = 1*10**-5

# sum_of_iterations = 0
# num_iterations = 10

# for it in range(0,num_iterations):
#     # print(f"-------------- ITERATION = {it+1} --------------")
#     P_i, mu_i, iteration = run_GaBP_HARDWARE_ACCELERATED(A, b, caching=True, node_updates_per_pe=NODE_UPDT_PE, number_pes=PEs, TRUE_MEAN=final_mean, max_iter=10000, mae=True, convergence_threshold=async_convergence_threshold, show=False)
#     sum_of_iterations += iteration
#     print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")

# print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")

#### Random (Exclusion) - No Cache

In [9]:
# async_convergence_threshold = 1*10**-5

# sum_of_iterations = 0
# num_iterations = 10

# for it in range(0,num_iterations):
#     # print(f"-------------- ITERATION = {it+1} --------------")
#     P_i, mu_i, iteration, _ = run_GaBP_HARDWARE_ACCELERATED_EXCLUSION(A, b, node_updates_per_pe=NODE_UPDT_PE, number_pes=PEs, TRUE_MEAN=final_mean, max_iter=10000, mae=True, convergence_threshold=async_convergence_threshold, show=False)
#     sum_of_iterations += iteration
#     # print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")

# print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")

#### Random (Exclusion) - With Cache

In [10]:
# async_convergence_threshold = 1*10**-5

# sum_of_iterations = 0
# num_iterations = 10

# for it in range(0,num_iterations):
#     # print(f"-------------- ITERATION = {it+1} --------------")
#     P_i, mu_i, iteration, _ = run_GaBP_HARDWARE_ACCELERATED_EXCLUSION(A, b, caching=True, node_updates_per_pe=NODE_UPDT_PE, number_pes=PEs, TRUE_MEAN=final_mean, max_iter=10000, mae=True, convergence_threshold=async_convergence_threshold, show=False)
#     sum_of_iterations += iteration
#     # print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")

# print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")

#### Residual - with Cache

In [11]:
# async_convergence_threshold = 1*10**-5

# sum_of_iterations = 0
# num_iterations = 10

# for it in range(0,num_iterations):
#     # print(f"-------------- ITERATION = {it+1} --------------")
#     P_i, mu_i, iteration = run_GaBP_HARDWARE_ACCELERATED_RESIDUAL(A, b, caching=False, node_updates_per_pe=NODE_UPDT_PE, number_pes=PEs, TRUE_MEAN=final_mean, max_iter=1000, mae=True, convergence_threshold=async_convergence_threshold, show=True)
#     sum_of_iterations += iteration
#     # print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")

# print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")

#### Residual - with Cache

In [12]:
# async_convergence_threshold = 1*10**-5

# sum_of_iterations = 0
# num_iterations = 10

# for it in range(0,num_iterations):
#     # print(f"-------------- ITERATION = {it+1} --------------")
#     P_i, mu_i, iteration = run_GaBP_HARDWARE_ACCELERATED_RESIDUAL(A, b, caching=True, node_updates_per_pe=NODE_UPDT_PE, number_pes=PEs, TRUE_MEAN=final_mean, max_iter=1000, mae=True, convergence_threshold=async_convergence_threshold, show=True)
#     sum_of_iterations += iteration
#     # print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")

# print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")

#### Fixed - No Caching

In [13]:
# async_convergence_threshold = 1*10**-5

# sum_of_iterations = 0
# num_iterations = 10

# for it in range(0,num_iterations):
#     node_update_schedule = np.arange(num_nodes, dtype=np.int64)
#     np.random.shuffle(node_update_schedule)
#     P_i, mu_i, iteration = run_GaBP_HARDWARE_ACCELERATED(A, b, caching=False, mode='fixed', node_update_schedule_enter=node_update_schedule, node_updates_per_pe=NODE_UPDT_PE, number_pes=PEs, TRUE_MEAN=final_mean, max_iter=1000, mae=True, convergence_threshold=async_convergence_threshold, show=False)
#     # print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")
#     # print(f"streams = {iteration}")
#     # print(f"node_update_schedule = {node_update_schedule}")
#     sum_of_iterations += iteration

# print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")

### Fixed - Cache

In [14]:
async_convergence_threshold = 1*10**-5

sum_of_iterations = 0
num_iterations = 10

for it in range(0,num_iterations):
    node_update_schedule = np.arange(num_nodes, dtype=np.int64)
    np.random.shuffle(node_update_schedule)
    P_i, mu_i, iteration = run_GaBP_HARDWARE_ACCELERATED(A, b, caching=True, mode='fixed', node_update_schedule_enter=node_update_schedule, node_updates_per_pe=NODE_UPDT_PE, number_pes=PEs, TRUE_MEAN=final_mean, max_iter=1000, mae=True, convergence_threshold=async_convergence_threshold, show=False)
    # print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")
    # print(f"streams = {iteration}")
    # print(f"node_update_schedule = {node_update_schedule}")
    sum_of_iterations += iteration

print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")

iteration: 0
2.6162409562987343
-----
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
iteration: 0
2.6162409562987343
-----
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
iteration: 0
2.6162409562987343
-----
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
iteration: 0
2.6162409562987343
-----
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
iteration: 0
2.6162409562987343
-----
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
iteration: 0
2.6162409562987343
-----
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
iteration: 0
2.616

### NO STREAMS (BESTCASE)

In [15]:
# async_convergence_threshold = 1*10**-5

# sum_of_iterations = 0
# num_iterations = 100

# for it in range(0,num_iterations):
#     # print(f"-------------- ITERATION = {it+1} --------------")
#     P_i, mu_i, iteration = run_GaBP_HARDWARE_BESTCASE(A, b, node_updates_per_stream=NODE_UPDT_PE, TRUE_MEAN=final_mean, max_iter=10000, mae=True, convergence_threshold=async_convergence_threshold, show=True)
#     sum_of_iterations += iteration
#     print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")

# print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")

### NO STREAMS (RESIDUAL)

In [16]:
# async_convergence_threshold = 1*10**-5

# sum_of_iterations = 0
# num_iterations = 100

# for it in range(0,num_iterations):
#     # print(f"-------------- ITERATION = {it+1} --------------")
#     P_i, mu_i, iteration = run_GaBP_HARDWARE_BESTCASE_RESIDUAL(A, b, node_updates_per_stream=NODE_UPDT_PE, TRUE_MEAN=final_mean, max_iter=500, mae=True, convergence_threshold=async_convergence_threshold, show=False)
#     sum_of_iterations += iteration
#     print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")

# print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")